In [3]:
import numpy as np
import matplotlib as plt
import csv
import os
import re
import pandas as pd


In [87]:
#A function to get file name and out put file names and legend labels
def getfilename():
    path = os.getcwd()    
    filewithext = []
    filename = []
    for file in [doc for doc in os.listdir(path) if doc.endswith('.DTA')]:  
        filewithext.append(file)
        filename.append(file.replace(".DTA",""))
    return(filewithext, filename)
#A function converts scientific notationed numbers to floats
def convert_to_float(s):
    try:
        s[x_tag] = float(s[x_tag])
        s[y_tag] = float(s[y_tag])
        return s
    except:
        return s

#A function return a columns of the result of a truth table
#which x_rag and y_tag column should both be numerical values
def is_numerical(s):
    try:
        result = isinstance(s[x_tag], float) and isinstance(s[y_tag], float)
        return result
    except:
        return False

In [88]:
df = pd.read_csv('lsv.txt')
df = df['XY'].str.split(r"\t", expand=True)

In [106]:

raw=df.iloc[1:,:]
# for index in range(raw.shape[1]):
#     print(raw[index])
#     raw[index].apply(pd.to_numeric, errors='ignore')
# type(raw.iloc[1][2])
#a = format(raw.iloc[1][2],float)
output = raw.iloc[1][2].str.format( f"{raw.iloc[1][2]:.9f}")

print(output)
#df.rename(columns = {0:'x1', 1:'y1', 2:'x2', 3:'y2', 4:'x3', 5:'y3'}, inplace = True)

AttributeError: 'str' object has no attribute 'str'

In [18]:
# filename_ls, legend_ls = getfilename()[0], getfilename()[1]
# x_tag = "Vf"
# y_tag = "Im"
# sample = 'RuZrOx_HER'
# filename_ls[0]
# with open('lsv1.DTA', 'r') as f:
#     for line in f.readlines():
#         print(line)
# #         if re.search('\tPt\tT.*',line):
# #             data_title = line
# # titles = data_title.split('\t')
# # titles[-1]=titles[-1].rstrip('\n')
# # titles[0] = 'CURVE_CYCLE'

TypeError: readlines() takes no keyword arguments

In [ ]:


#user input what x and y they want to use
filename_ls, legend_ls = getfilename()[0], getfilename()[1]
x_tag = "Vf"
y_tag = "Im"
# filename = 'cv50multi.DTA'
sample = 'RuZrOx_HER'

for filename in filename_ls:
    #get the titles of desired columns and replace the #1 \t column by 'CURVE_CYCLE'
    with open(filename, 'r') as f:
        for line in f.readlines():
            if re.search('\tPt\tT.*',line):
                data_title = line
    titles = data_title.split('\t')
    titles[-1]=titles[-1].rstrip('\n')
    titles[0] = 'CURVE_CYCLE'

    #read in data frame and fill the empty \t under CURVE_CYCLE as their curve name
    df = pd.read_csv(filename, sep = '\t', names= titles)
    df.CURVE_CYCLE.ffill(inplace=True)
    #Drop the empty entrys in x_tag and y_tag columns
    df.dropna(how = 'any', subset=[x_tag, y_tag], inplace=True)

    df[[x_tag, y_tag]] = df[[x_tag, y_tag]].apply(convert_to_float, axis= 1)
    num_df = df[df[[x_tag, y_tag]].apply(is_numerical, axis= 1)]
    num_df.reset_index(drop = True, inplace = True)

    for cycle in num_df.CURVE_CYCLE.unique():
        x, y = num_df[num_df.CURVE_CYCLE == cycle][x_tag], num_df[num_df.CURVE_CYCLE == cycle][y_tag] 
        plt.plot(x, y, label=cycle)

# current_handles, current_labels = plt.gca().get_legend_handles_labels()
# reversed_handles = list(reversed(current_handles))
# reversed_labels = list(reversed(current_labels))
plt.legend(legend_ls, frameon=False, handlelength=0)
# plt.xlim(1,20)
# plt.xticks(np.arange(1, 20, 2.0))
plt.yticks([])
plt.xlabel('Voltage (V vs. RHE)')
plt.ylabel('Current (A/cm$\mathregular{^{2}}$)')
plt.title('CV')
plt.savefig(sample + 'CV.png')
plt.show()